<a href="https://colab.research.google.com/github/WilsLoki/LLM/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install sentence_transformers
!pip install transformers
!pip install openai
!pip install unstructured
!pip install tiktoken
!pip install faiss-cpu
!pip install pypdf
!pip install -U langchain-community
!pip install -U langchain-huggingface

In [ ]:
import requests
import getpass
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

## Indexing

In [ ]:
# 创建知识库文档

# 下载文档
def download_file(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

download_file('https://wx.mail.qq.com/ftn/download?func=4&key=9ccd4666efb9e0a0aebe1966356163312ecb5e6637616331461d4f125457015054571a540e51541c015455561a035154521f045e005800035003075103507b310a5c43035b0d0a56065c434668020c5208425e121911075777cd605468779f379f5ad9323b22592b0d9288edaf6163316332376637&code=c27f7ac1', 'intelligent _cockpit.pdf')
download_file('https://wx.mail.qq.com/ftn/download?func=4&key=9a9c5238e1bbb5a9a8ef1b383b6336386f8d1f3839633638404c4d4c5f020f5a565b18010f555715565654011400000c534e535c5c560e0e010150085f02383804116a5f5502454b00101b485d0522187d5b418fa55b34ffda070c845a9b11a1ad78f83839633638656335&code=ec589c68', 'ar_glasses.pdf')

# 加载文档
loaders = [
    PyPDFLoader('ar_glasses.pdf'),
    PyPDFLoader('intelligent _cockpit.pdf')
]

# 存储文档
docs = []
for loader in loaders:
    docs.extend(loader.load())

# 验证：文档部分正常
print(f"文档数量: {len(docs)}")
print(f"第32个文档示例: {docs[32]}")

文档数量: 71
第32个文档示例: page_content='4
©2024.12 iResearchInc.                                                                                                                         www.iresearch.com.cn
AI大模型是什么？AI大模型作为人工智能领域的重要突破，其超大规模参数和超强计算资源使得机器能够处理海量数据，完成各种复杂任务AI大模型，通常指的是基于深度学习技术构建的、具有大量参数和强大功能的的人工智能模型。数据、算力和算法是AI大模型发展的三大核心要素。这些模型通过学习海量数据和深度神经网络的优化，在各种任务上取得了显著成果。
来源：专家访谈，公开资料，艾瑞消费研究院自主研究及绘制。
算法算力
数据
AI大模型是什么？通常指的是基于深度学习技术构建的、具有大量参数和强大功能的的人工智能模型。这些模型能够处理和理解大规模的数据，并在多种复杂的任务中表现出色。它们的作用类似于大脑，可以处理和分析大量数据。这些模型通过训练过程学习，以执行各种任务，让计算机获得类似人类的“思考”能力。核心三要素数据：AI大模型需要大规模的训练数据来支持其复杂的网络结构和参数学习。数据的数量越多，模型能够学习到的信息就越全面，在处理各种任务时表现出更高的泛化能力。算力：AI大模型的训练需要高性能的计算硬件支持，如GPU、TPU等，大算力能够加速模型的训练过程，使得复杂模型的训练在合理的时间内完成。算法：算法指的是一系列解决问题的步骤和规则，这些步骤和规则是模型处理数据、学习知识和完成任务的基础。' metadata={'producer': 'macOS 版本11.3（版号20E232） Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20241213025705Z00'00'", 'moddate': "D:20241213025705Z00'00'", 'source': 'intelligent _cockpit.pdf', 'total_pages': 42, 'page': 3, 'page_label': '4'}

In [ ]:
# 文本分割
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=30, separator='\n')
splits = text_splitter.split_documents(docs)

# 验证：文本分割正常
print(f"分割后的文档数量: {len(splits)}")
print(f"第101个分割示例: {splits[101]}")

分割后的文档数量: 213
第101个分割示例: page_content='来源：公开资料，艾瑞消费研究院自主研究及绘制。
阿里云推出的通用大模型，具备广泛的任务处理能力，包括文本生成、问答、翻译等。创意文案：根据产品介绍自动撰写营销文案，修改润色文章，生成直播带货剧本等办公助理：提供SWOT分析、PPT框架生成等学习助手：应用于学科试题生成、制定个性化学习路径规划等……
百度开发的人工智能大语言模型，具备文本生成、语音合成、多语言支持、实时翻译、智能断句和个性化设置等能力。内容创作：撰写商业计划、市场分析报告等商业文案多模态：支持图像生成、图像处理、语音识别、音频分类等逻辑推理：进行数学计算、常识推理、逻辑校验、代码纠错等……' metadata={'producer': 'macOS 版本11.3（版号20E232） Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20241213025705Z00'00'", 'moddate': "D:20241213025705Z00'00'", 'source': 'intelligent _cockpit.pdf', 'total_pages': 42, 'page': 8, 'page_label': '9'}


In [ ]:
# 文本嵌入
embeddings = HuggingFaceEmbeddings(model_name='moka-ai/m3e-base')

# 存储向量
vector_store = FAISS.from_documents(splits, embeddings)

## LLM (ERNIE-Speed-128K)

In [ ]:
# 定义类
class BaiduErnie:
    host = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/ernie-speed-128k"  # ERNIE-Speed-128K 模型请求地址

    def __init__(self):
        # 在运行时获取 client_id 和 client_secret
        self.client_id = getpass.getpass("输入 client_id: ")
        self.client_secret = getpass.getpass("输入 client_secret: ")
        self.get_access_token()

    # 定义获取 access token 方法(method)
    def get_access_token(self):
        url = f"https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={self.client_id}&client_secret={self.client_secret}"
        response = requests.get(url)

        if response.status_code == 200:
            self.access_token = response.json()["access_token"]
            return self.access_token
        else:
            raise Exception("获取 access_token 失败")

    # 定义 chat 方法(method)
    def chat(self, prompt: str, user_id: str = "default_user"):
        # 单轮对话方法（无上下文）
        url = f"{self.host}?access_token={self.access_token}"
        # 构建请求参数
        data = {
            "messages": [{"role": "user", "content": prompt}],
            "user_id": user_id
        }

        try:
            # 发送请求
            response = requests.post(url, json=data)
            # 检查响应状态码
            response.raise_for_status()
            # 响应处理
            result = response.json()
            return result.get("result", "")
        # 异常处理
        except requests.exceptions.RequestException as e:
            raise Exception(f"对话请求失败: {e}")

In [ ]:
# 创建实例对象
baidu_ernie = BaiduErnie()

# 验证：大模型调用正常
response = baidu_ernie.chat(prompt='你是哪家公司开发的什么大语言模型？')
print(response)

输入 client_id: ··········
输入 client_secret: ··········
您好，我是由百度公司开发的人工智能语言模型，我的中文名是文心一言，英文名是ERNIE Bot。如果您有任何问题或需要帮助，可以随时向我提问。希望我的回答能够真正为您带来帮助。


In [ ]:
# 定义 chat 函数（封装）
def chat(prompt):
    result = baidu_ernie.chat(prompt=prompt)
    return result

## Retrieval


In [ ]:
# 定义question和检索数量
question = '千悟大模型对鸿蒙座舱的提升有哪些'
K = 5

In [ ]:
# 检索 + 重排
docs_and_scores = vector_store.similarity_search_with_score(question, k=K)

# 打印检索结果
for doc, score in docs_and_scores:
    source = doc.metadata['source']
    content = doc.page_content
    print(f'来源 {source} 字数 {len(content)} 匹配度 {score:.2f}')
    print(content[:100] + '……')
    print('------------------')

来源 intelligent _cockpit.pdf 字数 256 匹配度 128.05
对座舱的提升提升智能语音交互体验：千悟大模型的加持下，智能座舱中的语音助手（小艺）对用户指令的理解和执行更加智能和高效。提升舱内人体姿态感知：千悟大模型提高座舱的高精度感知能力和多模态融合控车技术。实……
------------------
来源 intelligent _cockpit.pdf 字数 258 匹配度 130.92
32
©2024.12 iResearchInc.                                                                           ……
------------------
来源 intelligent _cockpit.pdf 字数 317 匹配度 141.94
l增强识别和纠错能力：语音助手能够更准确地理解用户的指令，识别用户声纹甚至复刻声纹，能够进行自动纠错语音指令，并提供正确的内容。l提升自适应聆听能力：语音助手能够听懂用户在表达过程中的犹豫停顿，自动地……
------------------
来源 intelligent _cockpit.pdf 字数 109 匹配度 144.55
来源：公开资料，艾瑞消费研究院自主研究及绘制。
千悟大模型华为智能汽车解决方案为鸿蒙座舱打造的专属于用车场景的融合感知决策大模型，深度集成了视觉感知、语音交互、车控传感器等多种技术，实现了多模态信息的……
------------------
来源 intelligent _cockpit.pdf 字数 67 匹配度 164.35
大模型技术已赋予座舱更加准确、流畅、自然的语音交互体验，未来随着AI Agent的应用，汽车智能座舱将获得更加情感化和拟人化的交互体验……
------------------


## Generation

In [ ]:
# 定义 predict 函数
def predict(question, docs_and_scores):

    # 生成上下文
    context = ''
    for doc in docs_and_scores:
        context += doc[0].page_content
        context += '\n'

    # 提示词增强
    prompt = f'你是一个专注于知识理解和解答的 AI 助手，请基于以下已知信息回答问题。你只需要回答和已知信息相关的问题，如果问题和已知信息不相关，你可以直接回答"不知道" 问题：{question} 已知信息:{context}'

    # 调用大模型获取结果
    result = chat(prompt)
    print(result)

In [ ]:
predict(question, docs_and_scores)

基于已知信息，千悟大模型对鸿蒙座舱的提升主要体现在以下几个方面：

1. 提升智能语音交互体验：通过千悟大模型的加持，智能座舱中的语音助手对用户指令的理解和执行更加智能和高效。
2. 提升舱内人体姿态感知：千悟大模型提高座舱的高精度感知能力和多模态融合控车技术，实现跨设备互联与无缝流转。
3. 实现个性化与智能化服务：千悟大模型通过深度学习和用户行为分析，能够为用户提供更加个性化的服务和建议。
4. 增强识别和纠错能力：语音助手能够更准确地理解用户的指令，识别用户声纹并进行自动纠错语音指令。
5. 提升自适应聆听能力：语音助手能够听懂用户在表达过程中的犹豫停顿，自动地延长聆听等待时间。
6. 融合感知决策：千悟大模型深度集成了视觉感知、语音交互、车控传感器等多种技术，实现了多模态信息的融合感知和综合决策。

由此可见，千悟大模型在提升鸿蒙座舱的智能交互体验、感知能力、个性化服务以及多模态融合决策等方面发挥了重要作用。
